In [ ]:
import os
import pandas as pd
from bertopic import BERTopic
from src import preprocess_data, extract_date_days, generate_topic_labels, save_result_topics

# Ukrainian news

Read data

In [2]:
file_path = os.path.join("..", "data.xlsx")

data = pd.read_excel(io = file_path,
                     sheet_name = "Україна")

**Analyze data**

In [3]:
data.head()

,Дата,Джерело,Заголовок,Опис,Посилання,Автор,Популярність джерела,Мова,Країна
0,2025-02-12 09:28:50,сайт тернополя 0352.ua,"""200 тисяч – одразу, а через рік – виїзд за ко...",Про зниження мобілізаційного віку розмови трив...,https://www.0352.ua/news/3901613/200-tisac-odr...,Дмитро Бондаренко,116319,uk,UA
1,2025-02-12 19:39:16,Gazeta.ua,"""30 000 на місяць"": Зеленський пояснив, наскіл...",Найбільша мобілізація в Україні відбулася у 20...,https://gazeta.ua/articles/life/_30-000-na-mis...,Максимчук Леонід,19872,uk,UA
2,2025-02-09 18:20:00,сайт тернополя 0352.ua,"""34 000 грн штрафу та позбавлення прав"": в Укр...",З'явилося попереджання для українців,https://www.0352.ua/news/3900464/34-000-grn-st...,Георгій Шевчук,116319,uk,UA
3,2025-02-11 14:30:00,Patriot of Ukraine,"""90% успіху Росії в Криму в 2014-му році - це ...",Головний сержант 126 ОБр ТрО 30-го корпусу мор...,https://patrioty.org.ua/politic/90-uspikhu-ros...,NaN,234013,uk,UA
4,2025-02-11 16:11:00,24 Канал,"""Є приклади успішних"": в ОСУВ ""Хортиця"" відпов...",ЗСУ проводять успішні контратаки на Покровсько...,https://24tv.ua/kontrataki-zsu-bilya-pokrovska...,Дмитро Усик,14630,uk,UA


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14577 entries, 0 to 14576
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Дата                  14577 non-null  object
 1   Джерело               14139 non-null  object
 2   Заголовок             14577 non-null  object
 3   Опис                  14573 non-null  object
 4   Посилання             14577 non-null  object
 5   Автор                 12802 non-null  object
 6   Популярність джерела  14577 non-null  int64 
 7   Мова                  14577 non-null  object
 8   Країна                14577 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.0+ MB


We have some news without any description, but all news contain the header.

In [5]:
data["Мова"].value_counts()

Мова
uk    14577
Name: count, dtype: int64

All ukrainian news are in Ukrainian. So we have to use BERTopic that can handle Ukrainian language.

# Train BERTopic model

In [6]:
topic_model = BERTopic(embedding_model = "paraphrase-multilingual-MiniLM-L12-v2") # Multilanguage BERTopic

Create texts for BERTopic as: 'Заголовок' + 'Опис'

In [7]:
texts = preprocess_data(data)

Create timestamps for BERTopic as date from 'Дата'

In [8]:
timestamps = extract_date_days(data)

In [9]:
set(timestamps)

{'2025-02-09',
 '2025-02-10',
 '2025-02-11',
 '2025-02-12',
 '2025-02-13',
 '2025-02-14',
 '2025-02-15',
 '2025-02-16'}

We have news only during 8 days.

Train model

In [ ]:
topics, probs = topic_model.fit_transform(texts)

Output results

In [11]:
topic_model.get_topic_info().head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4471,-1_на_та_про_не,"[на, та, про, не, як, до, україни, що, рф, від]","[""Живу за часом Трампа"": Келлог сказав, коли м..."
1,0,369,0_путіним_трамп_трампа_розмову,"[путіним, трамп, трампа, розмову, трампом, пут...",[Трамп провів телефонну розмову з Путіним щодо...
2,1,296,1_збили_shahed_ппо_ніч,"[збили, shahed, ппо, ніч, бпла, типу, вночі, з...",[Протиповітряний бій 16 лютого: ППО збила 95 у...
3,2,261,2_обстріляли_поранені_російські_внаслідок,"[обстріляли, поранені, російські, внаслідок, в...",[Росіяни за добу обстріляли 36 населених пункт...
4,3,258,3_суспільство_фото_главпост_порталі,"[суспільство, фото, главпост, порталі, медіа, ...",[В Україні кружляють безпілотники: названо нап...


We have more than 4000 outliers.

In [12]:
topic_model.visualize_topics()

We have similar topics (duplicates). For example, about sanctions against Poroshenko.

**Reduce the amount of topics.**

In [14]:
topic_model.reduce_topics(texts, nr_topics=50)

# Results

In [15]:
topic_model.get_topic_info().head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4471,-1_на_та_про_що,"[на, та, про, що, україни, не, до, для, за, рф]",[Віцепрезидент США не виключає відправлення ві...
1,0,3274,0_на_рф_та_росія,"[на, рф, та, росія, за, області, про, добу, лю...","[На фронті за добу - 261 боєзіткнення, на Покр..."
2,1,2128,1_сша_україни_не_зеленський,"[сша, україни, не, зеленський, що, україні, на...",[Зеленський: Україна готова говорити з США про...
3,2,975,2_дронів_дрони_бпла_ніч,"[дронів, дрони, бпла, ніч, ппо, по, лютого, си...",[Масштабна атака ворожих дронів: Україна збила...
4,3,516,3_санкції_порошенка_проти_рнбо,"[санкції, порошенка, проти, рнбо, суд, санкцій...",[Forbes: РНБО запровадила санкції проти Пороше...


Set custom names to topics.

In [16]:
all_topic_ids = topic_model.get_topic_info()["Topic"].tolist()

topic_ids = [topic_id for topic_id in all_topic_ids if topic_id != -1] #exclude outliers (-1)

labels = generate_topic_labels(topic_model, topic_ids)

topic_model.set_topic_labels(labels)

**Plot interconnections between topics**

In [17]:
topic_model.visualize_topics(custom_labels=True)

**Topics over time**

**Plot the most important topics over time**

In [18]:
topics_over_time = topic_model.topics_over_time(texts,
                                                timestamps,
                                                nr_bins=8) # we have news during 8 days

In [19]:
topic_model.visualize_topics_over_time(topics_over_time,
                                       top_n_topics=5,
                                       custom_labels=True)

**Save results**

In [21]:
path_to_save = os.path.join("..", "output", "res_ukrainian.csv")

save_result_topics(topic_model, path_to_save)